In [3]:
# -*- coding: utf-8 -*-
"""
4개 품목(마늘, 생강, 양배추, 양파) 기초통계량 일괄 출력
"""

import pandas as pd
from io import StringIO

# ---------- 1) CSV 로더 ----------
def load_csv_safe(path: str) -> pd.DataFrame:
    encodings = ["utf-8-sig", "cp949", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except Exception as e:
            last_err = e
            df = None
    if df is None:
        raise RuntimeError(f"CSV 로딩 실패: {path} (마지막 에러: {last_err})")

    # 컬럼명 정리
    df.columns = (
        df.columns.astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    # '거래일자' 같은 날짜 컬럼 자동 변환
    date_cols = [c for c in df.columns if "거래일자" in c]
    for dc in date_cols:
        df[dc] = pd.to_datetime(df[dc], errors="coerce")

    return df

# ---------- 2) 파일 불러오기 ----------
garlic  = load_csv_safe("garlic.csv")
ginger  = load_csv_safe("ginger.csv")
cabbage = load_csv_safe("cabbage.csv")
onion   = load_csv_safe("onion.csv")

data_dict = {
    "마늘(garlic)": garlic,
    "생강(ginger)": ginger,
    "양배추(cabbage)": cabbage,
    "양파(onion)": onion,
}

# ---------- 3) 결측치 요약 함수 ----------
def na_summary(df: pd.DataFrame) -> pd.DataFrame:
    na_cnt = df.isna().sum()
    na_ratio = (na_cnt / len(df)) * 100 if len(df) > 0 else 0
    return pd.DataFrame({
        "na_count": na_cnt,
        "na_ratio_%": na_ratio
    }).sort_values(by="na_count", ascending=False)

# ---------- 4) 루프 실행 ----------
for name, df in data_dict.items():
    print("=" * 100)
    print(f"[{name}] 데이터 요약")
    print("=" * 100)

    # info() 출력
    buf = StringIO()
    df.info(buf=buf)
    info_str = buf.getvalue()
    print("\n[info()]")
    print(info_str)

    # describe(all)
    print("\n[describe(include='all')]")
    try:
        desc = df.describe(include="all", datetime_is_numeric=True)
    except TypeError:
        desc = df.describe(include="all")
    print(desc)

    # 결측치 요약
    print("\n[결측치 요약]")
    print(na_summary(df))

    # 각 컬럼 value_counts Top10
    print("\n[value_counts(Top 10 by column)]")
    for col in df.columns:
        try:
            vc = df[col].value_counts(dropna=False).head(10)
            print(f"\n- {col} (Top 10)")
            print(vc)
        except Exception as e:
            print(f"- {col}: value_counts 불가 ({e})")

    print("\n\n")


[마늘(garlic)] 데이터 요약

[info()]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10962 entries, 0 to 10961
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   거래일자           10962 non-null  datetime64[ns]
 1   품목명            10962 non-null  object        
 2   반입량            1193 non-null   float64       
 3   금액             1200 non-null   float64       
 4   전년 반입량         677 non-null    float64       
 5   전년 반입량 증감률(%)  10962 non-null  float64       
 6   전년 금액          693 non-null    float64       
 7   전년 금액 증감률(%)   10962 non-null  float64       
 8   평년 반입량         1106 non-null   float64       
 9   평년 반입량 증감률(%)  10962 non-null  float64       
 10  평년 금액          1127 non-null   float64       
 11  평년 금액 증감률(%)   10962 non-null  float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 1.0+ MB


[describe(include='all')]
                       거래일자     품목명           반

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
from io import StringIO
from pathlib import Path

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

def load_csv_safe(path: str) -> pd.DataFrame:
    encodings = ["utf-8-sig", "cp949", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except Exception as e:
            last_err = e
            df = None
    if df is None:
        raise RuntimeError(f"CSV 로딩 실패: {path} (마지막 에러: {last_err})")
    df.columns = df.columns.astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
    for c in [c for c in df.columns if "거래일자" in c]:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

# 1) 파일 불러오기 (코드 실행 폴더와 같은 위치)
garlic  = load_csv_safe("garlic.csv")
ginger  = load_csv_safe("ginger.csv")
cabbage = load_csv_safe("cabbage.csv")
onion   = load_csv_safe("onion.csv")

data_dict = {
    "마늘(garlic)": garlic,
    "생강(ginger)": ginger,
    "양배추(cabbage)": cabbage,
    "양파(onion)": onion,
}

# 2) 유틸
def na_summary(df: pd.DataFrame) -> pd.DataFrame:
    na_cnt = df.isna().sum()
    na_ratio = (na_cnt / len(df) * 100) if len(df) > 0 else 0
    return pd.DataFrame({"na_count": na_cnt, "na_ratio_%": na_ratio}).sort_values("na_count", ascending=False)

# 3) 출력 제어
MAX_VC_COLS = 20   # value_counts 돌릴 최대 컬럼 수 (많으면 상위 일부만)
TOPN = 10          # value_counts 상위 N

report_dir = Path("./reports_basic_stats")
report_dir.mkdir(exist_ok=True)

print("=== 대상 품목:", list(data_dict.keys()))
for name, df in data_dict.items():
    print("\n" + "="*100)
    print(f"[START] {name} 요약 생성 (rows={len(df)}, cols={df.shape[1]})")
    print("="*100)

    try:
        # info 캡처
        buf = StringIO(); df.info(buf=buf); info_str = buf.getvalue()

        # describe (가벼운 콘솔용 요약)
        try:
            desc = df.describe(include="all", datetime_is_numeric=True)
        except TypeError:
            desc = df.describe(include="all")

        # 결측치 요약
        nas = na_summary(df)

        # value_counts: 콘솔은 상위 몇 개 컬럼만, 파일에는 전체
        cols_for_console = list(df.columns)[:MAX_VC_COLS]

        # 콘솔 요약 출력 (가볍게)
        print("[info()]")
        print(info_str.splitlines()[0])  # 첫 줄만
        print("[columns]", list(df.columns))
        print("[describe] (head)")
        print(desc.head(8))  # 너무 길면 head만
        print("[결측치 요약] (상위 10)")
        print(nas.head(10))

        print(f"[value_counts] (콘솔은 앞 {len(cols_for_console)}개 컬럼만, 각 Top{TOPN})")
        for col in cols_for_console:
            try:
                vc = df[col].value_counts(dropna=False).head(TOPN)
                print(f"\n- {col} (Top {TOPN})")
                print(vc)
            except Exception as e:
                print(f"- {col}: value_counts 불가 ({e})")

        # 파일 리포트(자세한 내용)
        report_path = report_dir / f"{name}_basic_report.txt"
        with report_path.open("w", encoding="utf-8") as f:
            f.write(f"[{name}] 데이터 요약\n")
            f.write("="*80 + "\n\n[info()]\n")
            f.write(info_str + "\n\n[describe(include='all')]\n")
            f.write(desc.to_string() + "\n\n[결측치 요약]\n")
            f.write(nas.to_string() + "\n\n[value_counts(Top {TOPN} by column)]\n")
            for col in df.columns:
                try:
                    vc = df[col].value_counts(dropna=False).head(TOPN)
                    f.write(f"\n- {col} (Top {TOPN})\n{vc.to_string()}\n")
                except Exception as e:
                    f.write(f"- {col}: value_counts 불가 ({e})\n")
        print(f"[DONE] {name} 리포트 저장: {report_path.resolve()}")

    except Exception as e:
        print(f"[ERROR] {name} 처리 중 에러: {e}")

print("\n=== 전체 완료 ===")


=== 대상 품목: ['마늘(garlic)', '생강(ginger)', '양배추(cabbage)', '양파(onion)']

[START] 마늘(garlic) 요약 생성 (rows=10962, cols=12)
[info()]
<class 'pandas.core.frame.DataFrame'>
[columns] ['거래일자', '품목명', '반입량', '금액', '전년 반입량', '전년 반입량 증감률(%)', '전년 금액', '전년 금액 증감률(%)', '평년 반입량', '평년 반입량 증감률(%)', '평년 금액', '평년 금액 증감률(%)']
[describe] (head)
                       거래일자     품목명            반입량            금액       전년 반입량  전년 반입량 증감률(%)         전년 금액  \
count                 10962   10962    1193.000000  1.200000e+03   677.000000    10962.00000  6.930000e+02   
unique                  NaN       3            NaN           NaN          NaN            NaN           NaN   
top                     NaN  저장형 난지            NaN           NaN          NaN            NaN           NaN   
freq                    NaN    3654            NaN           NaN          NaN            NaN           NaN   
mean    2020-06-29 12:00:00     NaN    6379.848282  1.764893e+07  6363.813885      121.71829  1.910272e+07   
min     2015-06